In [68]:
import pandas as pd
from nltk import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import pygal
from pygal.style import Style
from textblob import TextBlob

nltk.download('punkt')
nltk.download('stopwords')

data = pd.read_csv("csv/tweets.csv", sep = ',')

[nltk_data] Downloading package punkt to /home/nomdebrew/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nomdebrew/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [69]:
columns_to_use = ['created_str','text']
data = data[columns_to_use]
data = data.dropna(subset=['text'])

#convert created_str to datetime and drop hour, min, sec
data['created_str'] = pd.to_datetime(data['created_str'])
data.created_str = data.created_str.map(lambda x: x.replace(day=1, hour=0, minute=0, second=0))


clintonData = data[data['text'].str.contains('clinton|hillary', case=False)]
trumpData = data[data['text'].str.contains('trump|donald', case=False)]
bothData = data[data['text'].str.contains('clinton|hillary|trump|donald', case=False)]

#add polarity and subjectivity columns to dataframes
clintonData['polarity'] = clintonData['text'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
clintonData['subjectivity'] = clintonData['text'].apply(lambda tweet: TextBlob(tweet).sentiment.subjectivity)
trumpData['polarity'] = trumpData['text'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
trumpData['subjectivity'] = trumpData['text'].apply(lambda tweet: TextBlob(tweet).sentiment.subjectivity)
bothData['polarity'] = bothData['text'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
bothData['subjectivity'] = bothData['text'].apply(lambda tweet: TextBlob(tweet).sentiment.subjectivity)

df_C1 = clintonData.groupby('created_str').mean()
df_T1 = trumpData.groupby('created_str').mean()
df_B1 = bothData.groupby('created_str').mean()

print(data.shape)
print(clintonData.shape)
print(trumpData.shape)
print(bothData.shape)

/home/nomdebrew/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/nomdebrew/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/nomdebrew/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



(203430, 2)
(25555, 4)
(40403, 4)
(57521, 4)


/home/nomdebrew/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [70]:
#get dates back into column instead of being an index
df_C1.to_csv('csv/clinton_1.csv', sep=',')
df_C1 = pd.read_csv('csv/clinton_1.csv')

df_T1.to_csv('csv/trump_1.csv', sep=',')
df_T1 = pd.read_csv('csv/trump_1.csv')

df_B1.to_csv('csv/both_1.csv', sep=',')
df_B1 = pd.read_csv('csv/both_1.csv')

In [71]:
custom_style = Style(colors=('red','blue','purple'))
date_chart = pygal.Line(x_label_rotation=60, fill=False, style=custom_style)
date_chart.title = 'Polarity Over Time'
date_chart.x_labels = df_C1['created_str']
date_chart.add("trump", df_T1['polarity'])
date_chart.add("Clinton", df_C1['polarity'])
date_chart.add("Both", df_B1['polarity'])
date_chart.render_to_file('images/polarity_date.svg')

date_chart = pygal.Line(x_label_rotation=60, fill=False, style=custom_style)
date_chart.title = 'Subjectivity Over Time'
date_chart.x_labels = df_C1['created_str']
date_chart.add("trump", df_T1['subjectivity'])
date_chart.add("Clinton", df_C1['subjectivity'])
date_chart.add("Both", df_B1['subjectivity'])
date_chart.render_to_file('images/subjectivity_date.svg')

In [72]:
custom_style = Style(colors=('red',))
gauge = pygal.SolidGauge(inner_radius=0.70, style=custom_style, show_legend=False)
percent_formatter = lambda x: '{:.10g}%'.format(x)
gauge.title = 'trump'
gauge.value_formatter = percent_formatter
gauge.add('trump', [{'value': round(len(trumpData)/len(data)*100,1), 'max_value': 100}])
gauge.render_to_file('images/t_guage.svg')

custom_style = Style(colors=('blue',))
gauge = pygal.SolidGauge(inner_radius=0.70, style=custom_style, show_legend=False)
percent_formatter = lambda x: '{:.10g}%'.format(x)
gauge.title = 'Clinton'
gauge.value_formatter = percent_formatter
gauge.add('Clinton', [{'value': round(len(clintonData)/len(data)*100,1), 'max_value': 100}])
gauge.render_to_file('images/c_guage.svg')

custom_style = Style(colors=('purple',))
gauge = pygal.SolidGauge(inner_radius=0.70, style=custom_style, show_legend=False)
percent_formatter = lambda x: '{:.10g}%'.format(x)
gauge.title = 'Both'
gauge.value_formatter = percent_formatter
gauge.add('Both', [{'value': round(len(bothData)/len(data)*100,1), 'max_value': 100}])
gauge.render_to_file('images/b_guage.svg')

In [73]:
#reindex dataframes
clintonData = clintonData.reset_index(drop=True)
trumpData = trumpData.reset_index(drop=True)
bothData = bothData.reset_index(drop=True)

<h3>Example Tweets</h3>

In [74]:
clintonData = clintonData.sort_values(by=['polarity'])
print(clintonData.loc[0:19,['text','polarity']])

                                                    text  polarity
0      RT @NahBabyNah: Twitchy: Chuck Todd caught out...  0.000000
12243  Wikileaks 13: Emails Prove #Obama Lied When He...  0.000000
5445   #TopNews Obama turns focus to U.S. Congress as...  0.000000
5446   RT @TRUMPMOVEMENTUS: Thank you  America! - aga...  0.000000
5451   RT @sharronrenee26: #IStartCryingWhen @BernieS...  0.000000
5452   RT @nansen: EXCLUSIVE: Clinton Aides Resisted ...  0.000000
5453   RT @Tuniekc: #LEGACY\n5 yrs since Gaddafi’s ki...  0.000000
5454   Hillary just explained how Trump’s death would...  0.000000
5456   RT @pmatas: Top Clinton Aide: Loss 'Most Devas...  0.000000
5457   RT @ClintonCantBail: LA TIMES POLL: TRUMP 47.2...  0.000000
5458   RT @FakeTonyAbbott1: Cowardice of the highest ...  0.000000
5459   Rand Paul: Polls showing Clinton ahead are 'de...  0.000000
5462   RT @jojokejohn: ALL TROLL HILLARY OR TRUMP  OO...  0.000000
5463   RT @ZaibatsuNews: Yes, Deplorables, I'm Voting...  0.00

In [75]:
trumpData = trumpData.sort_values(by=['subjectivity'])
print(trumpData.loc[0:19,['text','subjectivity']])

                                                    text  subjectivity
0      RT @mcicero10: #BernieSanders #Trump people sh...           0.0
31651  RT @ChristiChat: MT @ChristiChat: #WakeUpAmeri...           0.0
15678  RT @SCLconservative: The State Controlled Medi...           0.0
15676  RT @FreedomChild3: Seven Ways Obama Is Trying ...           0.0
15675  RT @theglobaluniter: @realDonaldTrump \n\n💥 Wh...           0.0
15673  RT @Thiru_0914: 🇺🇸Donald J. Trump Rally SATURD...           0.0
15671  RT @Trump__Pence: VIDEO : Senator Tim Scott Pr...           0.0
31655  RT @business: Bloomberg exclusive: Vladimir Pu...           0.0
31650  RT @realDonaldTrump: Today in Florida, I pledg...           0.0
31657  Man is trying to scale Trump Tower in NYC usin...           0.0
31659  RT @GoForTimmer: #ItsUnacceptableTo Act like r...           0.0
31661  .@ericbolling: "Can you imagine the president-...           0.0
31662  RT @TrueCOT: Team Hillary Gave GOP Establishme...           0.0
31664 

<h3>Remove Stop Words From Tweets</h3>

In [90]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
custom_stopwords = ['RT','https','http','@']
stop.extend(custom_stopwords)

clintonData['without_stopwords'] = clintonData['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
trumpData['without_stopwords'] = trumpData['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
bothData['without_stopwords'] = bothData['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

<h3>Tokenize Tweets</h3>

In [91]:
clintonData['tokenized_sents'] = clintonData.apply(lambda row: nltk.word_tokenize(row['without_stopwords']), axis=1)
trumpData['tokenized_sents'] = trumpData.apply(lambda row: nltk.word_tokenize(row['without_stopwords']), axis=1)
bothData['tokenized_sents'] = bothData.apply(lambda row: nltk.word_tokenize(row['without_stopwords']), axis=1)

<h3>Lemmatize Tweets</h3>

In [92]:
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in text]

clintonData['text_lemmatized'] = clintonData.tokenized_sents.apply(lemmatize_text)
trumpData['text_lemmatized'] = trumpData.tokenized_sents.apply(lemmatize_text)
bothData['text_lemmatized'] = bothData.tokenized_sents.apply(lemmatize_text)

In [93]:
clintonData.head()

,created_str,text,polarity,subjectivity,without_stopwords,tokenized_sents,text_lemmatized
8237,2016-10-01,RT @luvGodncountry: Wikileaks: Clinton Adviser...,-1.0,1.0,@luvGodncountry: Wikileaks: Clinton Adviser Ta...,"[@, luvGodncountry, :, Wikileaks, :, Clinton, ...","[@, luvGodncountry, :, Wikileaks, :, Clinton, ..."
10990,2016-06-01,RT @andersonDrLJA: #BillClinton #Obama #Hillar...,-1.0,1.0,@andersonDrLJA: #BillClinton #Obama #Hillary A...,"[@, andersonDrLJA, :, #, BillClinton, #, Obama...","[@, andersonDrLJA, :, #, BillClinton, #, Obama..."
19887,2016-09-01,RT @venus58: Hey @ShepNewsTeam your show was A...,-1.0,0.8,@venus58: Hey @ShepNewsTeam show ALL CRAP 2day...,"[@, venus58, :, Hey, @, ShepNewsTeam, show, AL...","[@, venus58, :, Hey, @, ShepNewsTeam, show, AL..."
1049,2016-09-01,RT @Col_Connaughton: ASSANGE: CLINTON MEDIA 'E...,-1.0,1.0,@Col_Connaughton: ASSANGE: CLINTON MEDIA 'EREC...,"[@, Col_Connaughton, :, ASSANGE, :, CLINTON, M...","[@, Col_Connaughton, :, ASSANGE, :, CLINTON, M..."
8562,2016-10-01,"Tennessee GOP\n@TEN_GOP\nTrump: ""Hillary Clint...",-1.0,1.0,"Tennessee GOP @TEN_GOP Trump: ""Hillary Clinton...","[Tennessee, GOP, @, TEN_GOP, Trump, :, ``, Hil...","[Tennessee, GOP, @, TEN_GOP, Trump, :, ``, Hil..."
